# Limpieza de Datos: Carritos Abandonados en un E-commerce de Moda

Este notebook guía paso a paso la limpieza de un dataset de carritos abandonados en un e-commerce de moda chileno.
Los datos contienen errores intencionales que deben corregirse antes del análisis.


In [ ]:
import pandas as pd

# Cargar el dataset
file_path = "carritos_abandonados.csv"  # Asegúrate de cargar el archivo correcto
df = pd.read_csv(file_path)

# Mostrar las primeras filas
df.head()


## 1. Corregir valores negativos en `ValorCarrito`

In [ ]:
df["ValorCarrito"] = df["ValorCarrito"].abs()
df.head()

## 2. Manejar valores faltantes (`Genero`, `Region`, `Dispositivo`)

In [ ]:
df["Genero"].fillna("No especificado", inplace=True)
df["Region"].fillna(df["Region"].mode()[0], inplace=True)
df["Dispositivo"].fillna(df["Dispositivo"].mode()[0], inplace=True)
df.isnull().sum()

## 3. Corregir inconsistencias en `ClienteRegistrado` y `ClienteRecurrente`

In [ ]:
df.loc[(df["ClienteRegistrado"] == "No") & (df["ClienteRecurrente"] == "Sí"), "ClienteRegistrado"] = "Sí"
df.head()

## 4. Eliminar RUTs duplicados

In [ ]:
df = df.drop_duplicates(subset=["RUT"], keep="first")
df.shape

## 5. Corregir fechas imposibles (`FechaAbandono`)

In [ ]:
df["FechaAbandono"] = pd.to_datetime(df["FechaAbandono"], errors="coerce")
df = df.dropna(subset=["FechaAbandono"])
df.head()

## 6. Detectar valores atípicos en `ValorCarrito`

In [ ]:
q1 = df["ValorCarrito"].quantile(0.05)
q3 = df["ValorCarrito"].quantile(0.95)
df = df[(df["ValorCarrito"] >= q1) & (df["ValorCarrito"] <= q3)]
df.describe()

## 7. Corregir edades erróneas (`Edad`)

In [ ]:
df.loc[(df["Edad"] < 18) | (df["Edad"] > 75), "Edad"] = df["Edad"].median()
df.describe()

## 8. Revisar inconsistencias en `CantidadProductos`

In [ ]:
df["CantidadProductos"] = df["Productos"].apply(lambda x: len(str(x).split("|")))
df.head()

## 9. Corregir categorías mal asignadas

In [ ]:
correcciones_categorias = {"Vestido largo rojo": "Ropa mujer", "Zapatos de vestir marrón": "Calzado", "Chaqueta de cuero negra": "Ropa hombre"}
df["CategoriasPrincipales"] = df["Productos"].apply(lambda x: correcciones_categorias.get(x.split("|")[0], df["CategoriasPrincipales"]))
df.head()